In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./3.type.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(666):
            try:
                data = {'func': 'navnode_fetch',
                        'payload': '{"jsn":{"tab":"extras","groupname":"' + input_.loc[a, 'Category'] + '","subgroupname":"' + input_.loc[a, 'Subcategory'] + '","parttype":"' + input_.loc[a, 'Type_Code'] + '","nodetype":"parttype"}}',
                        'api_json_request': '1'}

                resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                     data=data,
                                     headers=UA.get_User_Agent_Requests(),
                                     proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                json_data = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(json_data['html_fill_sections']['navchildren[]'], 'lxml')
                html = etree.HTML(str(soup))
                
                # = = = = = = = = = = = = = = =
                
                list_part = html.xpath('//input[contains(@id, "listing_data_supplemental")]/@value')
                
                # = = = = = = = = = = = = = = =
                
                list_part_number = [json.loads(part)['partnumber'].strip() for part in list_part]
                
                # = = = = = = = = = = = = = = =
                
                list_manufacturer = [json.loads(part)['catalogname'].strip() for part in list_part]

                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame({'Tab_No.': 3,
                                        'Tab': 'Tools & Universal Parts',
                                        'Category_No.': int(input_.loc[a, 'Category_No.']),
                                        'Category': input_.loc[a, 'Category'],
                                        'Subcategory_No.': int(input_.loc[a, 'Subcategory_No.']),
                                        'Subcategory': input_.loc[a, 'Subcategory'],
                                        'Type_No.': int(input_.loc[a, 'Type_No.']),
                                        'Type': input_.loc[a, 'Type'],
                                        'Type_Code': input_.loc[a, 'Type_Code'],
                                        'Tab_Url' : input_.loc[a, 'Tab_Url'],
                                        'Part_No.': [i+1 for i in range(len(list_part))],
                                        'Manufacturer': list_manufacturer,
                                        'Part_Number': list_part_number})

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
                
                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Category_No.': int(input_.loc[a, 'Category_No.']),
                                     'Category': input_.loc[a, 'Category'],
                                     'Subcategory_No.': int(input_.loc[a, 'Subcategory_No.']),
                                     'Subcategory': input_.loc[a, 'Subcategory'],
                                     'Type_No.': int(input_.loc[a, 'Type_No.']),
                                     'Type': input_.loc[a, 'Type'],
                                     'Type_Code': input_.loc[a, 'Type_Code'],
                                     'Tab_Url' : input_.loc[a, 'Tab_Url']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Category'] + ' - ' + input_.loc[a, 'Subcategory'] + ' - ' + input_.loc[a, 'Type'] + '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Category_No.', 'Subcategory_No.', 'Type_No.', 'Part_No.'], ascending=[True, True, True, True]).reset_index(drop=True)
output_correct.to_excel('./4.part_number-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Category_No.', 'Subcategory_No.', 'Type_No.'], ascending=[True, True, True]).reset_index(drop=True)
    output_error.to_excel('./part_number_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：1765

[ok] - Apparel & Gifts - RockAuto Gift - Gift Certificate
[尝试次数：1] - [剩余数量：1745] - [当前时间：18:29:06]

[ok] - Apparel & Gifts - RockAuto Apparel - Hat
[尝试次数：1] - [剩余数量：1744] - [当前时间：18:29:06]

[ok] - Belt Drive - ATV / UTV Belt - 1 1/32"      (26.2mm) Belt
[尝试次数：1] - [剩余数量：1743] - [当前时间：18:29:06]

[ok] - Apparel & Gifts - RockAuto Gift - Lanyard
[尝试次数：1] - [剩余数量：1742] - [当前时间：18:29:06]

[ok] - Belt Drive - ATV / UTV Belt - 1 3/16"      (30.2mm) Belt
[尝试次数：1] - [剩余数量：1741] - [当前时间：18:29:06]

[ok] - Belt Drive - ATV / UTV Belt - <1"      (< 25.4mm) Belt
[尝试次数：1] - [剩余数量：1740] - [当前时间：18:29:06]

[ok] - Belt Drive - ATV / UTV Belt - 1 1/4"      (31.8mm) Belt
[尝试次数：1] - [剩余数量：1739] - [当前时间：18:29:06]

[ok] - Belt Drive - ATV / UTV Belt - 1 7/32"      (31mm) Belt
[尝试次数：1] - [剩余数量：1738] - [当前时间：18:29:06]

[ok] - Belt Drive - ATV / UTV Belt - 1 3/8"      (34.9mm) Belt
[尝试次数：1] - [剩余数量：1737] - [当前时间：18:29:06]

[ok] - Apparel & Gifts - RockAuto Gift - Magnets
[尝试次数：1] - [剩余数量：1736] - [当前时间

[ok] - Belt Drive - Snowmobile Belt - 1 5/16"        (33.3mm) Belt
[尝试次数：1] - [剩余数量：1670] - [当前时间：18:29:21]

[ok] - Belt Drive - Snowmobile Belt - 1 9/32"        (32.5mm) Belt
[尝试次数：1] - [剩余数量：1669] - [当前时间：18:29:21]

[ok] - Belt Drive - Refrigeration V-Belt - Smooth Belt
[尝试次数：1] - [剩余数量：1668] - [当前时间：18:29:21]

[ok] - Belt Drive - Tool - Belt Removal / Installation Tool
[尝试次数：1] - [剩余数量：1667] - [当前时间：18:29:21]

[ok] - Body & Lamp Assembly - Accessory Lighting - Beacon
[尝试次数：1] - [剩余数量：1666] - [当前时间：18:29:22]

[ok] - Body & Lamp Assembly - Accessory Lighting - Docking Light
[尝试次数：1] - [剩余数量：1665] - [当前时间：18:29:22]

[ok] - Body & Lamp Assembly - Accessory Lighting - Auxiliary Light
[尝试次数：1] - [剩余数量：1664] - [当前时间：18:29:22]

[ok] - Body & Lamp Assembly - Accessory Lighting - Flood / Spot Light
[尝试次数：1] - [剩余数量：1663] - [当前时间：18:29:22]

[ok] - Body & Lamp Assembly - Accessory Lighting - Identification Light Bar
[尝试次数：1] - [剩余数量：1662] - [当前时间：18:29:22]

[ok] - Body & Lamp Assembly - Accesso

[ok] - Body & Lamp Assembly - Clearance / Side Marker Lamp - ECE Compliant Marker Lamp
[尝试次数：1] - [剩余数量：1597] - [当前时间：18:29:30]

[ok] - Body & Lamp Assembly - Clearance / Side Marker Lamp - Housed Marker Lamp
[尝试次数：1] - [剩余数量：1596] - [当前时间：18:29:30]

[ok] - Body & Lamp Assembly - Clearance / Side Marker Lamp - Clearance/Side Marker Replacement Lens
[尝试次数：1] - [剩余数量：1595] - [当前时间：18:29:31]

[ok] - Body & Lamp Assembly - Cargo Management - Tailgate Lock
[尝试次数：1] - [剩余数量：1594] - [当前时间：18:29:31]

[ok] - Body & Lamp Assembly - Clearance / Side Marker Lamp - Mini Marker Lamp Lens
[尝试次数：1] - [剩余数量：1593] - [当前时间：18:29:31]

[ok] - Body & Lamp Assembly - Clearance / Side Marker Lamp - Oval LED Clearance & Side Marker Lamp
[尝试次数：1] - [剩余数量：1592] - [当前时间：18:29:31]

[ok] - Body & Lamp Assembly - Clearance / Side Marker Lamp - Hard Hat Clearance & Side Marker Lamp
[尝试次数：2] - [剩余数量：1591] - [当前时间：18:29:31]

[ok] - Body & Lamp Assembly - Accessory Lighting - Utility Light
[尝试次数：1] - [剩余数量：1590] - [当前时间

[ok] - Body & Lamp Assembly - Lift Support - 20"-25"+ Extended Length Lift Support
[尝试次数：2] - [剩余数量：1524] - [当前时间：18:29:45]

[ok] - Body & Lamp Assembly - Lift Support - 30" And Greater Extended Length Lift Support
[尝试次数：3] - [剩余数量：1523] - [当前时间：18:29:45]

[ok] - Body & Lamp Assembly - Lift Support - Lift Support Bracket
[尝试次数：3] - [剩余数量：1522] - [当前时间：18:29:45]

[ok] - Belt Drive - Heavy Duty / Industrial V-Belt - 21/32"     (16.7mm) Belt
[尝试次数：1] - [剩余数量：1521] - [当前时间：18:29:47]

[ok] - Body & Lamp Assembly - Lift Support - 17"-18"+ Extended Length Lift Support
[尝试次数：7] - [剩余数量：1520] - [当前时间：18:29:47]

[ok] - Body & Lamp Assembly - Lift Support - 14"+ Extended Length Lift Support
[尝试次数：5] - [剩余数量：1519] - [当前时间：18:29:47]

[ok] - Body & Lamp Assembly - Lift Support - 15"-16"+ Extended Length Lift Support
[尝试次数：8] - [剩余数量：1518] - [当前时间：18:29:48]

[ok] - Body & Lamp Assembly - Outside Mirror - West Coast Mirror
[尝试次数：1] - [剩余数量：1517] - [当前时间：18:29:48]

[ok] - Body & Lamp Assembly - Outside

[ok] - Body & Lamp Assembly - Polishing - Polisher Pad
[尝试次数：16] - [剩余数量：1450] - [当前时间：18:30:02]

[ok] - Body & Lamp Assembly - Stop, Turn, & Tail Lamp - Flush-Mount Stop, Turn, & Tail Lamp
[尝试次数：2] - [剩余数量：1449] - [当前时间：18:30:02]

[ok] - Body & Lamp Assembly - Stop, Turn, & Tail Lamp - Round 7" Stop, Turn, & Tail Lamp
[尝试次数：1] - [剩余数量：1448] - [当前时间：18:30:02]

[ok] - Body & Lamp Assembly - Stop, Turn, & Tail Lamp - Oval LED Stop, Turn, & Tail Lamp
[尝试次数：1] - [剩余数量：1447] - [当前时间：18:30:03]

[ok] - Body & Lamp Assembly - Retainer / Clip - Retainer / Clip
[尝试次数：9] - [剩余数量：1446] - [当前时间：18:30:03]

[ok] - Body & Lamp Assembly - Stop, Turn, & Tail Lamp - Stud-Mount Stop, Turn, & Tail Lamp
[尝试次数：1] - [剩余数量：1445] - [当前时间：18:30:03]

[ok] - Body & Lamp Assembly - Stop, Turn, & Tail Lamp - Round LED 4" Stop, Turn, & Tail Lamp
[尝试次数：2] - [剩余数量：1444] - [当前时间：18:30:03]

[ok] - Body & Lamp Assembly - Stop, Turn, & Tail Lamp - Surface Mount Stop, Turn & Tail Light
[尝试次数：1] - [剩余数量：1443] - [当前时间：18:30:0

...

[ok] - Hoses/Lines & Clamps - Coolant Hose - Coolant Hose (Straight) - Standard
[尝试次数：17] - [剩余数量：371] - [当前时间：18:34:04]

[ok] - Hoses/Lines & Clamps - Fitting - Connector - Bubble Flare Connector
[尝试次数：15] - [剩余数量：370] - [当前时间：18:34:05]

[ok] - Hoses/Lines & Clamps - Fitting - Nut - Double Compression Sleeve Nut
[尝试次数：1] - [剩余数量：369] - [当前时间：18:34:05]

[ok] - Hoses/Lines & Clamps - Fitting - Connector - Gladhand Connector
[尝试次数：15] - [剩余数量：368] - [当前时间：18:34:05]

[ok] - Hoses/Lines & Clamps - Fitting - Nut - Compression Nut
[尝试次数：1] - [剩余数量：367] - [当前时间：18:34:05]

[ok] - Hoses/Lines & Clamps - Fitting - Nut - Inverted Flare Tube Nut
[尝试次数：1] - [剩余数量：366] - [当前时间：18:34:06]

[ok] - Hoses/Lines & Clamps - Fitting - Plug - Inverted Flare Plug
[尝试次数：1] - [剩余数量：365] - [当前时间：18:34:06]

[ok] - Hoses/Lines & Clamps - Fitting - Nut - SAE 45º Flare Nut
[尝试次数：1] - [剩余数量：364] - [当前时间：18:34:06]

[ok] - Hoses/Lines & Clamps - Fitting - Elbow - Push-On Hose Elbow
[尝试次数：2] - [剩余数量：363] - [当前时间：18:34:0

[ok] - Hoses/Lines & Clamps - Fitting - End Cap - Plastic Connector Cap
[尝试次数：2] - [剩余数量：295] - [当前时间：18:34:21]

[ok] - Hoses/Lines & Clamps - Transmission Fluid Cooler Line / Hose - Transmission Fluid Cooler Line / Hose Clip
[尝试次数：2] - [剩余数量：294] - [当前时间：18:34:21]

[ok] - Hoses/Lines & Clamps - Vacuum - Vacuum Hose Connector
[尝试次数：1] - [剩余数量：293] - [当前时间：18:34:21]

[ok] - Hoses/Lines & Clamps - Tool - Hose / Tube Bender
[尝试次数：9] - [剩余数量：292] - [当前时间：18:34:22]

[ok] - Hoses/Lines & Clamps - Valve - Shut-Off Valve
[尝试次数：1] - [剩余数量：291] - [当前时间：18:34:22]

[ok] - Hoses/Lines & Clamps - Transmission Fluid Cooler Line / Hose - Transmission Fluid Cooler Line / Hose
[尝试次数：2] - [剩余数量：290] - [当前时间：18:34:22]

[ok] - Hoses/Lines & Clamps - Valve - Drain Cock
[尝试次数：1] - [剩余数量：289] - [当前时间：18:34:22]

[ok] - Ignition - Distributor - Distributor Advance Weight
[尝试次数：1] - [剩余数量：288] - [当前时间：18:34:22]

[ok] - Hoses/Lines & Clamps - Tool - Hose Removal Tool
[尝试次数：11] - [剩余数量：287] - [当前时间：18:34:22]

[ok]

[ok] - Interior - Gauge - Oil Pressure Gauge
[尝试次数：1] - [剩余数量：210] - [当前时间：18:34:43]

[ok] - Interior - Gauge - Gauge Panel
[尝试次数：1] - [剩余数量：209] - [当前时间：18:34:44]

[ok] - Interior - Fluid / Chemical - Interior Cleaner
[尝试次数：4] - [剩余数量：208] - [当前时间：18:34:44]

[ok] - Interior - Gauge - Coolant Temperature Gauge
[尝试次数：1] - [剩余数量：207] - [当前时间：18:34:44]

[ok] - Interior - Gauge - Fuel Level Gauge
[尝试次数：5] - [剩余数量：206] - [当前时间：18:34:45]

[ok] - Interior - Gauge - Speedometer
[尝试次数：1] - [剩余数量：205] - [当前时间：18:34:46]

[ok] - Interior - Gauge - Tachometer
[尝试次数：1] - [剩余数量：204] - [当前时间：18:34:47]

[ok] - Interior - Gauge - Tachometer Adapter
[尝试次数：1] - [剩余数量：203] - [当前时间：18:34:47]

[ok] - Interior - Gauge - Transmission Fluid Temperature Gauge
[尝试次数：1] - [剩余数量：202] - [当前时间：18:34:47]

[ok] - Interior - Dome Lamp - Dome Light Lens
[尝试次数：10] - [剩余数量：201] - [当前时间：18:34:47]

[ok] - Interior - Gauge - Vacuum Gauge
[尝试次数：1] - [剩余数量：200] - [当前时间：18:34:48]

[ok] - Interior - Gauge - Water Pressure Gauge
[

[ok] - Suspension - Leaf Spring - 3/8" Bolt
[尝试次数：3] - [剩余数量：121] - [当前时间：18:35:01]

[ok] - Steering - Power Steering - Power Steering Repair Kit
[尝试次数：2] - [剩余数量：120] - [当前时间：18:35:01]

[ok] - Suspension - Leaf Spring - 5/8" Bolt
[尝试次数：1] - [剩余数量：119] - [当前时间：18:35:01]

[ok] - Suspension - Leaf Spring - 1/2" Bolt
[尝试次数：4] - [剩余数量：118] - [当前时间：18:35:02]

[ok] - Steering - Tool - Pulley Tool
[尝试次数：7] - [剩余数量：117] - [当前时间：18:35:02]

[ok] - Suspension - Leaf Spring - Leaf Spring 3000+ lb Load Rating
[尝试次数：7] - [剩余数量：116] - [当前时间：18:35:03]

[ok] - Suspension - Fluid / Chemical - Hydraulic System Fluid
[尝试次数：4] - [剩余数量：115] - [当前时间：18:35:03]

[ok] - Suspension - Shock Absorber - Shock Bushing
[尝试次数：2] - [剩余数量：114] - [当前时间：18:35:03]

[ok] - Suspension - Leaf Spring - Leaf Spring Bushing
[尝试次数：7] - [剩余数量：113] - [当前时间：18:35:04]

[ok] - Suspension - Leaf Spring - Leaf Spring 1000-1950 lb Load Rating
[尝试次数：6] - [剩余数量：112] - [当前时间：18:35:04]

[ok] - Suspension - Leaf Spring - Leaf Spring Insulator

[ok] - Wheel - Tool - Tire Air Chuck
[尝试次数：7] - [剩余数量：35] - [当前时间：18:35:20]

[ok] - Wheel - Hardware - Wheel Washer
[尝试次数：17] - [剩余数量：34] - [当前时间：18:35:20]

[ok] - Wiper & Washer - Fluid / Chemical - Windshield Washer Fluid
[尝试次数：1] - [剩余数量：33] - [当前时间：18:35:21]

[ok] - Wheel - Tire - Tire Pressure Monitoring System (TPMS) Sensor & Tool Kit
[尝试次数：12] - [剩余数量：32] - [当前时间：18:35:21]

[ok] - Wheel - Tool - Tire Pressure Monitoring System (TPMS) Torque Tool
[尝试次数：8] - [剩余数量：31] - [当前时间：18:35:21]

[ok] - Wheel - Tire - Spare Tire Hold Down
[尝试次数：15] - [剩余数量：30] - [当前时间：18:35:21]

[ok] - Wiper & Washer - Windshield Washer - Windshield Washer Hose
[尝试次数：1] - [剩余数量：29] - [当前时间：18:35:21]

[ok] - Wiper & Washer - Windshield Washer - Windshield Washer Pump
[尝试次数：1] - [剩余数量：28] - [当前时间：18:35:21]

[ok] - Wiper & Washer - Windshield Washer - Windshield Washer Nozzle
[尝试次数：2] - [剩余数量：27] - [当前时间：18:35:21]

[ok] - Wiper & Washer - Wiper Arm - Wiper Arm Spring
[尝试次数：2] - [剩余数量：26] - [当前时间：18:35:22]

[ok